In [55]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from deepface import DeepFace

In [56]:

def extract_features(img_path):
  embedding_objs = DeepFace.represent(img_path, model_name='Facenet512', enforce_detection=False)
  return embedding_objs[0]['embedding']

In [57]:

from tqdm import tqdm

In [58]:
root = '/home/ubuntu/ai_attendance_system/deepface/data_face'
data = []
dirs = os.listdir(root)
print(dirs)
for dir in tqdm(dirs):
  for img in os.listdir(os.path.join(root, dir)):
     features = extract_features(os.path.join(root, dir, img))
     features.insert(0, dir)
     data.append(features)

['HTMVan.10B1', 'TCTai.10B1', 'PPTuan.10B1', 'NDTAnh.10B1', 'HDAKiet.10B1', 'TCNNam.10B1', 'PTTHa.10B1']


100%|██████████| 7/7 [00:12<00:00,  1.72s/it]


In [59]:
column_names = ['person name']
for i in range(512):
  column_names.append(f'f{i+1}')

df_org = pd.DataFrame(data, columns=column_names)
df_org.to_csv('face_features.csv', index=False)
df_org.head()

,person name,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f503,f504,f505,f506,f507,f508,f509,f510,f511,f512
0,HTMVan.10B1,0.322058,-0.433161,-0.672233,0.686782,-0.239042,0.528821,-0.515339,0.269433,-0.235005,...,-0.153020,0.075565,-0.272968,0.273718,0.294176,0.196406,0.295513,0.115156,0.034416,-0.239489
1,HTMVan.10B1,-0.799124,0.649721,-1.067423,1.807344,0.254739,-0.177150,-0.582705,0.131899,-0.022124,...,-0.000305,-0.628453,-1.339539,1.494982,-0.879591,0.565622,1.095937,-0.650479,0.758743,-0.012012
2,HTMVan.10B1,-0.494682,0.468054,-1.165669,1.400342,0.095167,-0.615882,-0.052469,-0.175951,-0.743183,...,0.024124,-0.445072,-1.593938,1.555155,-1.084509,0.877635,1.105216,-0.701744,0.424702,0.208243
3,HTMVan.10B1,0.045172,-0.712837,-1.211775,0.617426,-0.491368,-0.414576,-0.231331,0.170432,0.065418,...,-0.674945,1.091405,-0.310416,0.427969,0.120300,-0.435138,-0.545808,0.412314,0.128153,-0.313113
4,HTMVan.10B1,-0.703672,0.678344,-1.159584,1.581358,0.286007,-0.368840,-0.169128,-0.048448,-0.908420,...,0.229358,-0.605996,-1.726386,1.940127,-0.327482,1.084144,0.538159,-1.262533,0.400448,0.192428


In [60]:
df = df_org.replace(dirs, [i for i in range(len(dirs))])
df = df.sample(frac=1)
df.head()

,person name,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f503,f504,f505,f506,f507,f508,f509,f510,f511,f512
18,3,0.415207,-0.386182,-0.761581,1.765350,-0.089976,0.127776,-0.878439,0.434871,0.394741,...,0.533462,-1.782721,-1.069523,1.544035,-0.665312,0.594126,-0.130272,-1.246574,-0.258193,-0.434187
29,5,1.269337,0.391684,0.524789,2.250469,1.596831,0.101942,-0.982712,-0.360430,0.127809,...,-0.836020,-0.980137,-0.671515,1.548209,0.084967,1.098884,0.479823,-1.366727,1.329335,-0.765459
31,6,0.065413,-0.187295,-1.038659,1.538978,-0.293485,0.406746,-0.348509,0.684315,-0.318287,...,0.325161,0.523756,-0.721655,-0.184646,-0.015526,0.533285,0.616137,-0.227958,-0.162661,-0.050137
30,6,1.190615,-0.271401,-0.603150,1.718566,-0.408453,-0.944755,-0.986922,0.457718,-0.744641,...,-2.107196,-0.692395,-0.864722,0.875340,-0.775757,-0.057353,-0.884720,-1.016677,2.120405,-0.331296
23,4,-0.297778,0.712079,-1.237908,1.002707,-0.435174,-0.874092,-0.535254,0.903722,0.565182,...,-0.610860,-0.238660,-0.984327,-0.053598,-1.631189,1.065352,0.878357,-1.644465,0.052931,-0.355536


In [61]:

cut = round(len(df) * .8)

x_train = df.iloc[ :cut , 1:]
y_train = df['person name'][ :cut]
x_test = df.iloc[cut: , 1:]
y_test = df['person name'][cut: ]
     

In [62]:
len(x_train)

28

In [63]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, 'tanh', input_shape=(512,)),
    tf.keras.layers.Dense(256, 'relu'),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dense(len(dirs), 'softmax')
])
model.compile(tf.keras.optimizers.Adamax(.0001), 'sparse_categorical_crossentropy', metrics=['accuracy'])
output = model.fit(x_train, y_train, epochs=60, validation_data=(x_test, y_test))

Epoch 1/60
1/1 [==============================] - 1s 578ms/step - loss: 1.9301 - accuracy: 0.2500 - val_loss: 1.8190 - val_accuracy: 0.0000e+00
Epoch 2/60
1/1 [==============================] - 0s 28ms/step - loss: 1.6433 - accuracy: 0.4286 - val_loss: 1.7393 - val_accuracy: 0.2857
Epoch 3/60
1/1 [==============================] - 0s 35ms/step - loss: 1.4405 - accuracy: 0.6429 - val_loss: 1.6877 - val_accuracy: 0.4286
Epoch 4/60
1/1 [==============================] - 0s 45ms/step - loss: 1.2877 - accuracy: 0.7500 - val_loss: 1.6448 - val_accuracy: 0.4286
Epoch 5/60
1/1 [==============================] - 0s 40ms/step - loss: 1.1635 - accuracy: 0.7857 - val_loss: 1.6112 - val_accuracy: 0.4286
Epoch 6/60
1/1 [==============================] - 0s 47ms/step - loss: 1.0610 - accuracy: 0.8571 - val_loss: 1.5823 - val_accuracy: 0.4286
Epoch 7/60
1/1 [==============================] - 0s 44ms/step - loss: 0.9739 - accuracy: 0.8571 - val_loss: 1.5574 - val_accuracy: 0.4286
Epoch 8/60
1/1 [======

In [64]:
model.save('weights/model.h5')

In [65]:
feature_vector1 = extract_features('kiet.jpg')

In [66]:
prediction = model.predict([feature_vector1])

# Get the index of the class with the highest probability
predicted_class = np.argmax(prediction)

# Get the confidence level (probability) of the predicted class
confidence = prediction[0][predicted_class] * 100 
print(f"The predicted class is {predicted_class} with a confidence of {confidence:.2f}%")
print(dirs[np.argmax(model.predict([feature_vector1]))])

1/1 [==============================] - 0s 49ms/step
The predicted class is 4 with a confidence of 57.65%
1/1 [==============================] - 0s 23ms/step
HDAKiet.10B1
